<a href="https://colab.research.google.com/github/afdmoraes/GEOSelper/blob/main/Semana_3_Aula_1_Manipulacao_de_Imagens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso de Programação para Sensoriamento Remoto
---

* Gilberto Ribeiro de Queiroz
* Thales Sehn Körting

## Tópicos desta aula

* Manipulação de Imagens com a Biblioteca GDAL



## Estrutura de armazenamento

Uma imagem (comumente chamada de *raster*) pode ser armazenada de diferentes formas, uma delas é através do armazenamento individual: um raster independente para cada banda. Neste caso, cada arquivo possui metadados (sistema de coordenadas, limites geográficos, dimensões), independentes. Veja por exemplo as imagens do CBERS-04A (Figura 1):
  * CBERS_4A_WPM_20200612_200_139_L4_BAND1.tif (Banda 1 - azul)
  * CBERS_4A_WPM_20200612_200_139_L4_BAND2.tif (Banda 2 - verde)
  * CBERS_4A_WPM_20200612_200_139_L4_BAND3.tif (Banda 3 - vermelho)

![picture](https://drive.google.com/uc?id=19XAC4P_B_FlJ8TwZUu1y86F1yn_vPkLq)

Figura 1 - Um raster independente para cada banda.

**Obs.:** A associação de cores na Figura 1 acima é apenas ilustrativa.

Outra maneira é armazenar diversas bandas no mesmo arquivo. Neste caso, o conjunto de metadados vale para todas as bandas. Por exemplo, é possível gerar um raster contendo as bandas 1, 2, 3 e 4 do CBERS-04A em um único arquivo GeoTIFF (Figura 2):
  * CBERS_4A_WPM_20200612_200_139_L4_BANDS1234.tif

![picture](https://drive.google.com/uc?id=16WVvfHfM3nxRxEcRskvwLRvkRbfFE8It)

**Figura 2** - Um arquivo com múltiplas bandas.

**Obs.:** Um raster pode ter qualquer número de bandas, desde que suportado pelo seu formato.

## Grade

A grade (ou *grid*) contém metadados relacionados às imagens, incluindo:
  * limites geográficos
  * sistema de referência espacial
  * número de linhas/colunas
  * 6 coeficientes para transformação afim:
    * [0] coordenada $x$ da célula do canto superior esquerdo (*upper left cell*) do raster.
    * [1] Resolução do pixel ao longo do eixo-$x$ .
    * [2] rotação em $x$ (0 se o raster for alinhado ao norte).
    * [3] coordenada $y$ da célula do canto superior esquerdo (*upper left cell*) do raster.
    * [4] rotação em $y$ (0 se o raster for alinhado ao norte).
    * [5] Resolução do pixel ao longo do eixo-$y$ .

## Banda  

A banda (*band*) contém as informações dos níveis digitais das imagens, além de outras propriedades:
  * NoDataValue
  * Minimum/Maximum
  * Histogram
  * Tipo de dados
  * Estatísticas (média/desvio padrão)
  * matriz de pixels

# GDAL - **G**eospatial **D**ata **A**bstraction **L**ibrary
---

A `GDAL` é uma biblioteca de software livre que fornece uma camada de abstração de dados geoespaciais que possibilita o desenvolvimento de aplicações que manipulam dados nos mais diferentes formatos e sistemas. A API (*Application Programming Interface* ou Interface de Programação de Aplicações) desta biblioteca encontra-se disponível para uso em Python através de um *binding* (ou *wrapper*), que fornece acesso às funcionalidades implementadas em C++. 


A GDAL é basicamente composta de quatro APIs:

* `GDAL`: Voltada para manipulação de dados matriciais (raster), com capacidade de leitura e escrita de [diversos formatos de imagem de sensoriamento remoto](http://gdal.org/formats_list.html), como GeoTIFF, HDF, e JPEG, entre outros. Esta parte da API contém objetos para manipulação das dimensões de uma imagem, para acesso de leitura e escrita de blocos das bandas espectrais de uma imagem, acesso a metadados e manipulação de pirâmides de multi-resolução.

<br/>

* `OGR`: Parte da API voltada para manipulação de dados em [formatos vetoriais](https://gdal.org/drivers/vector/index.html), tais como ESRI Shapefile, Google KML e GeoJSON. Apresenta os conceitos de camada de informação, feições, atributos alfanuméricos e geométricos.

<br/>

* `OSR`: Voltada para a manipulação de projeções e sistemas de referência espacial.

<br/>

* `GNM`: Acrônimo de Geographic Network Model, esta parte da API serve ao propósito de manipulação de redes geográficas.

<br/>

As aplicações que utilizam a GDAL acessam todos os formatos suportados por ela através de um único modelo de dados abstrato. Além disso, a GDAL também conta com uma variedade de programas utilitários de linha de comando para a tradução de formatos e alguns tipos básicos de processamento.

<br/>

Nesta aula, utilizaremos a GDAL para manipulação de imagens de sensoriamento remoto. Faremos acesso aos valores dos pixels das bandas, além da criação de matrizes no formato `numpy` (que será o nosso próximo tópico do curso). 

## Importando a Biblioteca GDAL

Para acessar a API `gdal`, que permitirá manipular as imagens, devemos importar a biblioteca `osgeo`, como mostrado abaixo:

In [ ]:
from osgeo import gdal

gdal.UseExceptions()

Repare, na célula de código acima, que além de importar a API `gdal` na primeira linha, ativamos o uso de exceções nas operações da biblioteca GDAL (método `UseExceptions` na segunda linha).

**Atenção:** Esta aula foi elaborada com a versão `2.2.3` da `gdal`, o Python `3.6.9`, o Jupyter `4.5.0` e Jupyter Notebook `5.2.2`. Para saber a versão dessas ferramentas no seu ambiente de trabalho, use os comandos mostrados abaixo:

In [ ]:
print (gdal.__version__)

In [ ]:
!python --version

In [ ]:
!jupyter --version

In [ ]:
!jupyter notebook --version

**Dica:** Dentro de um [Jupyter Notebook](https://jupyter-notebook.readthedocs.io/en/stable/) podemos utilizar o recurso de auto-completar. Após o operador de membro "`.`", o ambiente Jupyter irá fornecer uma lista das operações disponíveis:

In [ ]:
gdal.

Também podemos obter ajuda sobre as funções e objetos disponíveis em uma biblioteca utilizando o caracter `?` logo após o nome para o qual desejamos consultar a ajuda, como mostrado abaixo para a função `Open`:

In [ ]:
gdal.Open?

**Dica:** Vocês verão que a documentação de bibliotecas como a Matplotlib e Numpy funciona muito bem no ambiente Jupyter.

## Abertura de um arquivo raster

A função `Open` é utilizada para abrir um *conjunto de dados* (ou *dataset*), que exige dois parâmetros:
  * **Nome do Arquivo**: caminho e nome completo

  * **Forma de Acesso**: constante que indica se o arquivo será usado apenas para leitura (`GA_ReadOnly`) ou se também será utilizado em operações de escrita (`GA_Update`).

<br/>

Faça o download do arquivo [crop_rapideye.tif](https://drive.google.com/file/d/1WbQ3aYWjYPpHxA3Z03_qVVTAncgr7PV_/view?usp=sharing).

<br/>

Em seguida, copie este arquivo para a área de dados do seu ambiente do *Google Colab*. Esta área encontra-se disponível nas opções da barra de ferramentas do lado esquerdo da janela principal do *Google Colab*, como indicado na Figura 3 abaixo.

![picture](https://drive.google.com/uc?id=1IcJHh8ulQl6CCQG_twxsdAbqXann5IbX)

**Figura 3** - Acesso à pasta de dados do *Google Colab*.

<br/>

Ao selecionar o ícone com a figura de uma pasta, uma área como a mostrada abaixo será apresentada (Figura 4):

![picture](https://drive.google.com/uc?id=1LygXiWEvH80E8PjXOmgHQyZEFwi-NmBp)

**Figura 4** - Pasta de dados do *Google Colab*.

<br/>


Para copiar o arquivo [crop_rapideye.tif](https://drive.google.com/file/d/1WbQ3aYWjYPpHxA3Z03_qVVTAncgr7PV_/view?usp=sharing), basta arrastar e soltar este arquivo sobre a área apresentada, como mostrado na Figura 5 abaixo:

![picture](https://drive.google.com/uc?id=16_f0A6Aa_Lhqu7b0rBapQDd2Mt_MMUDK)

**Figura 5** - Arquivo [crop_rapideye.tif](https://drive.google.com/file/d/1WbQ3aYWjYPpHxA3Z03_qVVTAncgr7PV_/view?usp=sharing) copiado para a pasta de dados do *Google Colab*.

<br/>

**Atenção:** A área de dados será apagada assim que o ambiente do Google Colab for encerrado.

Para abrir o arquivo `GeoTIFF` com a imagem, vamos utilizar a operação `Open` como mostrado abaixo:

In [ ]:
dataset = gdal.Open("crop_rapideye.tif", gdal.GA_ReadOnly)

In [ ]:
type(dataset)

osgeo.gdal.Dataset

## Estrutura do *Dataset*

### Sistema de Referência Espacial

Para saber qual o sistema de coordenadas de um *Dataset*, deve ser utilizado o método `GetProjectionRef`, que retorna uma descrição no formato `WKT` (*Well-Known Text*). O `WKT` é um formato textual, padronizado pela OGC, para representação de sistemas de referência espacial dos objetos geográficos.

<br/>

A célula de código abaixo mostra como recuperar essa informação a partir do método  `GetProjectionRef`.

In [ ]:
dataset.GetProjectionRef()

'PROJCS["WGS 84 / UTM zone 21S",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-57],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",10000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32721"]]'

Um Sistema de Referência Espacial (*Spatial Reference System* - SRS) ou sistema de coordenadas de referência (*Coordinate Reference System* - CRS) pode ser um sistema local, regional ou global usado para localizar objetos geográficos.

<br/>

Para permitir uma maior interoperabilidade e facilidade na utilização, vários sistemas de informação geográfica fazem referência a um Sistema de Referência Espacial indicando apenas um número inteiro que representa o **SRID** ou códigos, como o **EPSG**, que são definidos por autoridades como a Associação Internacional de Produtores de Petróleo e Gás.

<br/>

Para identificar os códigos corretos para o sistema de referência espacial do seu interesse, veja os seguintes portais:

* http://epsg.io

* http://spatialreference.org 

### Transformação Afim

O método `GetGeoTransform` retorna uma tupla com os *06 coeficientes* referentes à *transformação afim* do *dataset*, isto é, os parâmetros para transformação do espaço de coordenadas da imagem (linha e coluna) para coordendas georreferenciadas (coordenadas projetadas ou geográficas).

<br/>

Vamos obter o valor desses coeficientes:

In [ ]:
GT = dataset.GetGeoTransform()

print(GT)

(508810.0, 5.0, 0.0, 7857490.0, 0.0, -5.0)


Na tupla acima, temos o seguinte:

| Índice | Coeficiente | Descrição  |
|---|---|---|
| 0 | 508810.0 | Coordenada-$x$ do pixel do canto superior esquerdo da imagem. |
| 1 | 5.0 | Resolução do pixel ao longo do eixo-$x$. |
| 2 | 0.0 | Rotação ao longo das linhas. Zero para imagens alinhadas ao norte (*north-up image*). |
| 3 | 7857490.0 | Coordenada-$y$ do pixel do canto superior esquerdo da imagem. |
| 4 | 0.0 | Rotação ao longo das colunas. Zero para imagens alinhadas ao norte (*north-up image*) |
| 5 | -5.0 | Resolução do pixel ao longo do eixo-$y$. |

Os parâmetros acima podem ser usados na seguinte equação de transformação:

$$
\begin{cases}
X_{geo} = GT[0] + Coluna * GT[1] + Linha * GT[2]\\
Y_{geo} = GT[3] + Coluna * GT[4] + Linha * GT[5]
\end{cases}
$$

<br/>

No caso de imagens alinhadas ao norte (*north up images*), essa equação se resume a:

$$
\begin{cases}
X_{geo} = GT[0] + Coluna * GT[1]\\
Y_{geo} = GT[3] + Linha * GT[5]
\end{cases}
$$


Vamos calcular a localização no espaço geográfico do pixel da (coluna 20, linha 30):

In [ ]:
coluna = 20
linha = 30

x = GT[0] + coluna * GT[1]
y = GT[3] + linha * GT[5]

print(x, y)

508910.0 7857340.0


**Nota:** Para mais informaçõs sobre essa operação, consulte o seguinte tutorial da GDAL: [Geotransform Tutorial](https://gdal.org/tutorials/geotransforms_tut.html).

### Dimensões (Número de linhas e colunas)

Para saber o número de linhas e colunas do *dataset* que está sendo utilizado, devemos utilizar as propriedades `RasterYSize` e `RasterXSize` conforme pode ser visto no exemplo abaixo:

In [ ]:
linhas = dataset.RasterYSize
colunas = dataset.RasterXSize

print ("Número de linhas:", linhas)
print ("Número de colunas:", colunas)

### Bandas

Para saber o número de bandas de um *Dataset*, podemos utilizar a propriedade `RasterCount`, como indicado abaixo:

In [ ]:
dataset.RasterCount

5

Como pode ser observado na saída acima, o arquivo `crop_rapideye.tif` possui 5 bandas espectrais.

<br/>

**Atenção:** A GDAL numera as bandas de $1$ até $n$, onde $n$ é o número total de bandas contidas no `Dataset`. Lembre-se que os objetos do Python, como listas e tuplas, são indexados a partir do número $0$ até $n - 1$.

<br/>

No caso da amostra de imagem RapidEye, as bandas `5` e `3` correspondem às bandas `NIR` e `RED`. Para acessar os dados dessas bandas, devemos utilizar o método `GetRasterBand`, que retornará um objeto capaz de manipular os dados de uma dada banda. A célula abaixo mostra como utilizar essa operação.

In [ ]:
banda_nir = dataset.GetRasterBand(5)
banda_red = dataset.GetRasterBand(3)

Uma propriedade importante para manipulação correta dos dados é o tipo do dado armazenado e que pode ser lido a partir da banda.

In [ ]:
print ("Tipos de dados:")
print (" - banda NIR:", gdal.GetDataTypeName(banda_nir.DataType))
print (" - banda RED:", gdal.GetDataTypeName(banda_red.DataType))

Tipos de dados:
 - banda NIR: UInt16
 - banda RED: UInt16


É possível saber quais os valores mínimo e máximo dos pixels de uma banda utilizando o método `ComputeRasterMinMax`:

In [ ]:
menor_valor, maior_valor = banda_red.ComputeRasterMinMax()

print("Menor valor de RED:", menor_valor)
print("Maior valor de RED:", maior_valor)

Menor valor de RED: 0.0
Maior valor de RED: 20693.0


### Leitura dos dados de uma banda

Depois de criado o objeto banda (`banda_nir` e `banda_red`), podemos ler os dados dessa banda para iniciar qualquer processamento com estes valores. Para isso, utilizaremos o método `ReadAsArray`, que permitirá obter os dados da banda em uma matriz **NumPy**.

In [ ]:
matriz_red = banda_red.ReadAsArray()

Essa operação retornará uma matriz do `Numpy`:

In [ ]:
type(matriz_red)

In [ ]:
matriz_red

Agora, podemos utilizar todas as operações disponíveis na `NumPy`:

* Para verificar se a matriz gerada com a leitura da banda foi criada com o número de linhas e colunas de forma correta, podemos utilizar o atributo `shape`:

In [ ]:
matriz_red.shape

* Para saber o tipos de dados das células da matriz, podemos usar a propriedade `dtype`:

In [ ]:
matriz_red.dtype

Vamos obter uma matriz `NumPy` para a banda `NIR`:

In [ ]:
matriz_nir = banda_nir.ReadAsArray()

Agora, vamos computar um índice de vegetação, o `NDVI`, a partir das matrizes com os valores das bandas `RED` e `NIR`:

In [ ]:
matriz_red = matriz_red.astype(float)
matriz_nir = matriz_nir.astype(float)

# geracao de array derivado das bandas
matriz_ndvi = (matriz_nir - matriz_red) / \
              (matriz_nir + matriz_red + 0.000000001)

# mostrar as dimensoes e tipo de dado da matriz de saida
print(matriz_ndvi.shape)
print(matriz_ndvi.dtype)

In [ ]:
matriz_ndvi


Podemos combinar as bibliotecas **NumPy** e **Matplotlib** para visualizar as matrizes como imagens.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 8))

plt.subplot(131)
plt.title("Banda RED")
plt.imshow(matriz_red, cmap='gray');

plt.subplot(132)
plt.title("Banda NIR")
plt.imshow(matriz_nir, cmap='gray');

plt.subplot(133)
plt.title("NDVI")
plt.imshow(matriz_ndvi, cmap='gray', vmin=-1.0, vmax=1.0);

### Liberando um *Dataset*

Para liberar a memória e fechar o arquivo aberto, associe o valor `None` ao identificador do `dataset`:

In [ ]:
dataset = None 

# Considerações Finais
---

Na próxima aula iremos ver em detalhes o funcionamento das bibliotecas `NumPy` e `Matplotlib`. Ao trabalhar com processamento de imagens, geralmente combinamos as bibliotecas GDAL e NumPy para realizar novas operações, e GDAL e Matplotlib para construir as visualizações.